# Ekstrakcja Źródeł

<img src="https://live.staticflickr.com/65535/54443002259_4a8e1249dd_b.jpg" alt="Embedded Photo" width="500">

*Obraz wygenerowany za pomocą ChatGPT.*

## Wstęp

Modele językowe bywają skłonne do mówienia nieprawdy lub półprawdy, a także zmyślania faktów bez podawania źródeł. Obecnie coraz częściej używane są systemy, które zamiast odpowiadać na pytania bezpośrednio, wpierw przeszukują bazę danych, np. zbiór dokumentów, i dopiero na podstawie najlepiej pasujących dokumentów generują odpowiedź. Taka odpowiedź ma większe szanse być oparta na rzeczywistości i może być zweryfikowana przez człowieka -- o ile poprawnie odnalezione zostały właściwe źródła.

Oczywiście, źródeł może być bardzo dużo, więc metody przeszukiwania muszą być efektywne -- przetworzenie wszystkiego "na raz" bezpośrednio modelem językowym nie wchodzi w grę! W tym zadaniu skupisz się na znajdowaniu najlepszych źródeł dla zadanego zdania, korzystając z metody **embeddingów** (pl. zanurzeń wektorów).

Wyobraź sobie, że jesteś inżynierem AI w firmie opracowującej narzędzie do weryfikacji faktów naukowych. Twoim zadaniem jest stworzenie modułu, który potrafi szybko i skutecznie odnajdywać wiarygodne publikacje naukowe potwierdzające lub obalające konkretne stwierdzenia. Dzięki Twojemu rozwiązaniu, naukowcy, dziennikarze i decydenci będą mogli weryfikować informacje w oparciu o solidne podstawy naukowe, co jest szczególnie istotne w dobie dezinformacji.


## Zadanie

Twoim zadaniem jest opracowanie systemu, który generuje wysokiej jakości wektorowe reprezentacje (embeddingi) zarówno dla zapytań, jak i dokumentów źródłowych, umożliwiające precyzyjne dopasowanie właściwych źródeł do zapytań.

Mając do dyspozycji **queries** (pl. zbiór zapytań; zapytania na które szukamy źródeł) oraz **corpus** (pl. baza dokumentów/źródeł; zbiór rozważanych dokumentów), musisz zaimplementować funkcje, które przypisują zapytaniom oraz źródłom wektory liczb rzeczywistych o wymiarze $768$. Te wektory będą użyte do znalezienia źródeł dla każdego zapytania przez dostarczoną przez nas funkcję ewaluacyjną, która dla danego zapytania, ze zbioru dokumentów wybiera $k=10$ najbliższych sąsiadów (ang. $k$-Nearest Neighbours).

W rozwiązaniu możesz skorzystać z dostarczonego modelu bazującego na architekturze GPT2, który został specjalnie dotrenowany, aby był pomocny w otrzymywaniu dobrej jakości embeddingów.

W trakcie pracy nad rozwiązaniem będziesz mógł testować jego skuteczność na zbiorze walidacyjnym, który pozwoli Ci ocenić jakość generowanych embeddingów w kontekście zadania wyszukiwania właściwych dokumentów źródłowych.

### Dane

Dostępne dla Ciebie w tym zadaniu dane to:

- Zbiór zapytań (queries), dla których należy znaleźć odpowiednie źródła
- Korpus dokumentów (corpus), zawierający publikacje naukowe, które mogą być źródłami dla zapytań
- Informacje o dopasowaniu zapytań do dokumentów w zbiorze walidacyjnym

Twoje rozwiązanie będzie oceniane na benchmarku *SciFact*. Służy on do oceny systemów wyszukiwania i weryfikacji faktów w kontekście naukowym. Składa się z zestawu stwierdzeń (ang. queries) opartych na rzeczywistych publikacjach naukowych, a baza dokumentów (ang. corpus) to publikacje z zakresu nauk przyrodniczych i medycznych. Do każdego stwierdzenia istnieje co najmniej jedna publikacja, która je popiera lub obala. Dostarczamy kod służący do ładowania danych, więc dane opisujemy tu wyłącznie informacyjnie.


**Plik `corpus.jsonl`** zawiera unikalne identyfikatory, tytuły i streszczenia prac naukowych

Przykład pojedynczego dokumentu:
```
{
    "text_id": 13734012,
    "title": "Prevalent abnormal prion protein in human appendixes after bovine spongiform encephalopathy epizootic: large scale survey",
    "text": "OBJECTIVES To carry out a further survey (...) CONCLUSIONS This study corroborates previous studies and suggests a high prevalence of infection with abnormal PrP, indicating vCJD carrier status in the population compared with the 177 vCJD cases to date. These findings have important implications for the management of blood and blood products and for the handling of surgical instruments."
}
```

**Plik `queries_val.jsonl`** zawiera treści stwierdzeń oraz identyfikator pasującego tekstu źródłowego. Zbiór testowy, na których finalnie będzie oceniane Twoje rozwiązanie **nie będzie zawierał** identyfikatorów pasujących tekstów źródłowych.

Przykład pojedynczego zapytania:
```
{
    "query": "1 in 5 million in UK have abnormal PrP positivity.",
    "matching_text_id": 13734012
}
```

### Kryterium Oceny
Zaimplementowane przez Ciebie metody (funkcje) `Embedder.encode_queries` oraz `Embedder.encode_corpus` zostaną wykorzystane aby przetworzyć odpowiednio zapytania $q \in Q$ a także dokumenty $d \in C$ na wektory. W dalszej części będziemy wymiennie używać $q$ i $d$ zarówno w kontekście tekstów jak i ich embeddingów.

Załóżmy, że zapytaniu $q\in Q$ odpowiada złoty dokument $d\in C$.
Kod ewaluacyjny sortuje wszystkie dokumenty według odległości od $q$, otrzymując dokumenty $K_1, K_2, ..., K_n$, tak że $K_1$ jest najbliżej. Następnie oznaczamy jako $I$, indeks złotego dokumentu $d$ w tym ciągu. To znaczy, że $I - 1$ jest liczbą dokumentów, których odległość od $q$ jest mniejsza, niż odległość $q$ od $d$.

Odległość między wektorami liczymy za pomocą podobieństwa cosinusowego (ang. cosine similarity), które dla wektorów $v, w \in \mathbb{R}^n$ jest określone jako $\frac{v^Tw}{||v|| \cdot ||w||}$, gdzie $||v||$ to długość wektora $v$.

Wynik dla zapytania $q$ określamy jako  

$$\text{nDCG@10}(q) = \begin{cases}
\frac{1}{\log_2(I + 1)} & \text{jeśli $I \leq 10$} \\
0 & \text{w przeciwnym wypadku.}
\end{cases}$$

Czyli, im bliżej złoty dokument został umieszczony zapytania względem innych dokumentów, tym wyższy wynik -- jeśli 10 "złych" dokumentów jest bliżej zapytania to wynik za ten przykład to 0.

Ostatecznie ocena Twojego rozwiązania będzie opierać się na metryce **nDCG@10**, obliczanej jako średnia wartość tej metryki dla wszystkich zapytań $(q \in Q )$.

- Jeśli wynik **nDCG@10** będzie **niższy niż 0.2**, otrzymasz **0 punktów**.  
- Jeśli wynik **przekroczy 0.5**, otrzymasz **maksymalną liczbę punktów**, czyli **100**.  

Punktacja dla wartości pomiędzy tymi progami będzie naliczana proporcjonalnie.


## Ograniczenia

- Twoje rozwiazanie będzie testowane na Platformie Konkursowej bez dostępu do internetu oraz w środowisku z GPU.
- Ewaluacja Twojego finalnego rozwiązania na Platformie Konkursowej nie może trwać dłużej niż 10 minut z GPU.
- Embedding każdego zapytania oraz tekstu powinien mieć wymiar 768
- Lista dopuszczalnych bibliotek: `torch`, `pandas`, `numpy`, `nltk`, `transformers`.

## Pliki Zgłoszeniowe

Należy przesłać tylko ten notebook uzupełniony o Twoje rozwiązanie (patrz klasa `Embedder`).

## Wskazówki

- Model GPT2 jest modelem językowym typu dekoder. Modele typu dekoder działają tak, że dla danego ciągu tokenów (np. prefiksu przetwarzanego zdania) $t_1, t_2, \dots, t_n$ wyliczają ukryty wektor $h_{n+1} \in \mathbb{R}^d$, a następnie transformują go jedną ze swoich macierzy z wagami na $p_{n+1} \in \mathbb{R}^m$ -- rozkład prawdopodobieństwa na tokenach w słowniku.
- W porównaniu z dostępnym czasem wykonania, dokumentów jest wiele.

## Ewaluacja

Podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`.

Za to zadanie możesz zdobyć pomiędzy 0 a 100 punktów. Liczba punktów, którą zdobędziesz, będzie wyliczona na (tajnym) zbiorze testowym na Platformie Konkursowej na podstawie wyżej wspomnianego wzoru, zaokrąglona do liczby całkowitej. Jeśli Twoje rozwiązanie nie będzie spełniało powyższych kryteriów lub nie będzie wykonywać się prawidłowo, otrzymasz za zadanie 0 punktów.

# Kod Startowy

W tej sekcji inicjalizujemy środowisko poprzez zaimportowanie potrzebnych bibliotek i funkcji. Przygotowany kod tokenizatora, ładowania danych i ewaluacji ulatwi Ci operowanie na danych i pozwoli rozwiązać zadanie.

In [1]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

FINAL_EVALUATION_MODE = False  # W czasie sprawdzania Twojego rozwiązania, zmienimy tą wartość na True

In [2]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

import json
import os
from math import log2

import torch
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Tokenizer:
    def __init__(self, tokenizer_path, length=150):
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.padding_side = "right"
        self.length = length

    def __call__(self, batch_text):
        batch_tensor = self.tokenizer(
            batch_text,
            max_length=self.length,
            truncation=True,
            padding=True,
            return_tensors="pt"
        )
        return batch_tensor.to(device)

## Ładowanie Danych
W tej części zadania załadujemy dane treningowe.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

def load_corpus(file):
    corpus = {}
    with open(file, encoding="utf8") as f_in:
        for line in f_in:
            line = json.loads(line)
            corpus[line.get("text_id")] = {
                "text": line.get("text"),
                "title": line.get("title"),
            }
    return corpus

def load_queries(file):
    queries = {}
    matching_texts = {}
    with open(file, encoding="utf8") as f_in:
        for query_num, line in enumerate(f_in):
            line = json.loads(line)

            queries[query_num] = line.get("query")
            matching_texts[query_num] = line.get("matching_text_id")
    return queries, matching_texts

corpus = load_corpus("corpus.jsonl")
queries, matching_texts = load_queries("queries_val.jsonl")

print(f"Loaded {len(corpus)} texts and {len(queries)} queries.")

## Kod z Kryterium Oceniającym

Kod, zbliżony do poniższego, będzie używany do oceny rozwiązania na zbiorze testowym.

In [4]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

def evaluate_retrieval_ndcg(
    golden_matches: dict[int, int],
    results: dict[int, dict[int, float]],
) -> float:
    """
    Oblicza wartość metryki nDCG dla podanych wyników wyszukiwania.

    Funkcja oblicza wynik Twojego rozwiązania, bazując na wynikach dla $top\\_k$ najlepszych dokumentów według Twojego embeddera.

    :param golden_matches: Słownik ze złotymi przyporządkowaniami, gdzie kluczem jest id zapytania, a wartością jest id właściwego dokumentu.
    :param results: Słownik z wynikami wyszukiwania, gdzie kluczem jest id zapytania, a wartością jest słownik z id dokumentów i ich podobieństwami do danego zapytania.
    :return: Wartość metryki nDCG."""

    for query_id, v in results.items():
        results[query_id] = {k: v for k, v in sorted(v.items(), key=lambda item: -item[1])}

    ndcg_sum = 0
    for query_id, v in results.items():
        golden_document = golden_matches[query_id]
        for i, document_id in enumerate(v.keys()):
            if golden_document == document_id:
                ndcg_sum += 1 / log2(i + 2)

    ndcg = round(ndcg_sum / len(results), 5)
    return ndcg


def compute_score(ndcg: float) -> float:
    """
    Oblicza wynik punktowy na podstawie wartości metryki nDCG.
    """
    lower_bound = 0.2
    upper_bound = 0.5

    if ndcg <= lower_bound:
        return 0
    elif lower_bound < ndcg < upper_bound:
        return int(round(100 * (ndcg - lower_bound) / (upper_bound - lower_bound)))
    else:
        return 100

### Przeszukiwanie
Poniżej jest kod, który służy do wybierania dla danego zapytania $top\_k$ najlepszych dokumentów z korpusu.

In [5]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

def cos_sim(a: torch.Tensor, b: torch.Tensor):
    """
    Computes the cosine similarity cos_sim(a[i], b[j]) for all i and j.
    :return: Matrix with res[i][j]  = cos_sim(a[i], b[j])
    """
    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.mm(a_norm, b_norm.transpose(0, 1))

def search_topk_texts(
    embedder,
    corpus: dict[str, dict[str, str]],
    queries: dict[str, str],
    top_k: int = 10,
) -> dict[str, dict[str, float]]:
    results = {}

    # Create embeddings for all queries using model.encode_queries()
    # Runs semantic search against the corpus embeddings
    # Returns a ranked list with the corpus ids
    query_ids = list(queries.keys())
    results = {qid: {} for qid in query_ids}
    queries = [queries[qid] for qid in queries]
    query_embeddings = embedder.encode_queries(queries)

    corpus_ids = sorted(
        corpus,
        key=lambda k: len(corpus[k].get("title", "") + corpus[k].get("text", "")),
        reverse=True,
    )
    corpus = [corpus[cid] for cid in corpus_ids]

    # Encode chunk of corpus
    corpus_embeddings = embedder.encode_corpus(corpus)

    # Compute similarites using cosine-similarity
    cos_scores = cos_sim(query_embeddings, corpus_embeddings)
    cos_scores[torch.isnan(cos_scores)] = -1

    # Get top-k values
    cos_scores_top_k_values, cos_scores_top_k_idx = torch.topk(
        cos_scores,
        min(top_k + 1, len(cos_scores[1])),
        dim=1,
        largest=True,
        sorted=False,
    )
    cos_scores_top_k_values = cos_scores_top_k_values.cpu().tolist()
    cos_scores_top_k_idx = cos_scores_top_k_idx.cpu().tolist()

    for query_itr in range(len(query_embeddings)):
        query_id = query_ids[query_itr]
        for score, corpus_id in zip(cos_scores_top_k_values[query_itr], cos_scores_top_k_idx[query_itr]):
            results[query_id][corpus_ids[corpus_id]] = score

    return results

# Twoje Rozwiązanie
W tej sekcji należy umieścić Twoje rozwiązanie. Wprowadzaj zmiany wyłącznie tutaj!

In [6]:
class Embedder:
    # Nie zmieniaj sygnatury konstruktora
    def __init__(self):
        # TODO: możesz zmieniać tą metodę,
        # ale nie zmieniaj jej sygnatury! (tzn. nie zmieniaj argumentów)
        self.model = AutoModel.from_pretrained("Muennighoff/SGPT-125M-weightedmean-msmarco-specb-bitfit")
        self.tokenizer = Tokenizer("Muennighoff/SGPT-125M-weightedmean-msmarco-specb-bitfit")

    def encode_queries(self, queries: list[str]):
        """
        Funkcja kodująca zapytania.
        :param queries: Lista zapytań do zakodowania
        :return: Embeddingi zapytań - tensor o wymiarach (n, 768), gdzie n = len(queries) to liczba zapytań.
        """

        # TODO: zaimplementuj tę metodę - zakoduj zapytania
        # Nie zmieniaj sygnatury tej metody! (tzn. nie zmieniaj argumentów)
        # Pamiętaj, że możesz korzystać z modelu HuggingFace gpt-2...
        # Możesz użyć Tokenizatora, zaimplementowanego w górnej części notatnika
        # Wskazówka: Ewalaucja będzie szybsza jeśli zwracany tensor będzie znajdował się na gpu.
        ...
        return torch.ones(len(queries), 768).to(device)

    def encode_corpus(self, texts: list[dict]):
        """
        Funkcja kodująca teksty źródłowe.
        :param texts: Lista tekstów do zakodowania. Każdy tekst jest reprezentowany jako słownik:
            {
                "title": "..."
                "text": "...",
            }
        :return: Embeddingi tekstów - tensor o wymiarach (m, 768), gdzie m = len(texts) to liczba tekstów
        """

        # TODO: zaimplementuj tę metodę - zakoduj teksty źródłowe
        # Nie zmieniaj sygnatury tej metody! (tzn. nie zmieniaj argumentów)
        ...
        return torch.ones(len(texts), 768).to(device)

# Ewaluacja

Uruchomienie poniższej komórki pozwoli sprawdzić, ile punktów zdobyłoby Twoje rozwiązanie na danych walidacyjnych. Przed wysłaniem upewnij się, że cały notebook wykonuje się od początku do końca bez błędów i bez konieczności ingerencji użytkownika po wybraniu opcji "Run All".

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

if not FINAL_EVALUATION_MODE:
    embedder = Embedder()

    with torch.no_grad():
        results = search_topk_texts(embedder, corpus, queries, top_k=10)

    # Obliczenie nDCG
    ndcg = evaluate_retrieval_ndcg(matching_texts, results)

    # Obliczenie końcowego wyniku na podstawie nDCG
    points = compute_score(ndcg)

    print(f"\nLiczba zapytań: {len(queries)}")
    print(f"Liczba tekstów: {len(corpus)}")
    print(f"nDCG: {ndcg:.3f}")
    print(f"Wynik punktowy: {points}")


Podczas sprawdzania model zostanie zapisany jako `your_model.pkl` i oceniony na zbiorze testowym.

In [8]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

if FINAL_EVALUATION_MODE:
    import cloudpickle

    OUTPUT_PATH = "file_output"
    FUNCTION_FILENAME = "your_model.pkl"
    FUNCTION_OUTPUT_PATH = os.path.join(OUTPUT_PATH, FUNCTION_FILENAME)

    if not os.path.exists(OUTPUT_PATH):
        os.makedirs(OUTPUT_PATH)

    with open(FUNCTION_OUTPUT_PATH, "wb") as f:
        cloudpickle.dump(Embedder, f)